This is for a kaggle competition: https://www.kaggle.com/c/bosch-production-line-performance

This part that produces default_xgboost_results.csv has the highest accuracy right now.

In [ ]:
import xgboost
import pandas
import time
import io
import zipfile
import numpy
print('imports complete')

In [ ]:
# FYI: This takes about a minute and a half
def load_and_unzip(filename):
    print('loading and unzipping large training data file %s' % (filename))
    t0 = time.time()
    df = pandas.read_csv(filename, compression='zip')
    t1 = time.time()
    print('completed in %.3f seconds' % (t1-t0))
    return df

#train_numeric_filename = '../resources/train_numeric.csv.zip'
#df_explore = load_and_unzip(train_numeric_filename)
#test_numeric_filename = '../resources/test_numeric.csv.zip'
#df_test = load_and_unzip(test_numeric_filename)

Fill in the empty data with 0. get the mean and std of the entire training dataset, minus the id colum and the dependent variable column

In [ ]:
df_explore = pandas.read_csv('../resources/split_train_numeric_a.csv')
df_explore.fillna(0,inplace=True)
df_explore_meta = df_explore[df_explore.keys()[1:-1]].describe()
df_explore_meta

In [ ]:
def load_training_DMatrix(filename):
    arr = numpy.genfromtxt(filename, delimiter=',', skip_header=True, filling_values=0)
    return xgboost.DMatrix(arr[:,1:-1], label=arr[:,-1])
# load the testing file, without labels
def load_testing_DMatrix(filename):
    arr = numpy.genfromtxt(filename, delimiter=',', skip_header=True, filling_values=0)
    return arr[:,0], xgboost.DMatrix(arr[:,1:])

In [ ]:
dtrain = load_training_DMatrix('../resources/train_numeric.csv')
param = {'max_depth':2, 'eta':1, 'slient':1, 'objective':'binary:logistic'}
num_round = 10
bst = xgboost.train(param, dtrain, num_round)

In [ ]:
ids, dtest = load_testing_DMatrix('../resources/test_numeric.csv')

In [ ]:
pred = bst.predict(dtest)

In [ ]:
with open('default_xgboost_results.csv', 'w') as f:
    f.write('Id,Response\n')
    for i, p in enumerate(pred):
        f.write('%d,%s\n'%(ids[i],'1' if p > 0.5 else '0'))

use the xgboost classifier
cross validation with k fold
grid search over a few parameters
data stored in pandas DataFrame
still only using just the numeric data

In [ ]:
import sklearn.metrics
import sklearn.grid_search
import sklearn.cross_validation
import xgboost.sklearn
import pandas

In [ ]:
data = pandas.read_csv('../resources/train_numeric.csv')
data.fillna(0,inplace=True)
target_name = 'Response'
target = data[target_name]
features = data.drop(target_name, axis=1)

In [ ]:
def perf_metric(y_true, y_predict):
    return sklearn.metrics.matthews_corrcoef(y_true, y_predict)

def fit_model(X,y):
    cv_sets = sklearn.cross_validation.ShuffleSplit(X.shape[0], n_iter=10, test_size=0.1, random_state=0)
    clf = xgboost.sklearn.XGBClassifier(nthread=14)
    params = {
                'max_depth' : [6],
                'reg_lambda' : [1]
             }
    scoring_fnc = sklearn.metrics.make_scorer(perf_metric)
    grid = sklearn.grid_search.GridSearchCV(clf, params, scoring=scoring_fnc, cv=cv_sets, verbose=2)
    grid = grid.fit(X,y)
    return grid.best_estimator_

clf = fit_model(features, target)

In [ ]:
clf.get_params()

In [ ]:
test_data = pandas.read_csv('../resources/test_numeric.csv')
pred = clf.predict(test_data)
with open('xgboost_all_numeric_results.csv', 'w') as f:
    f.write('Id,Response\n')
    for i, p in enumerate(pred):
        f.write('%d,%d\n'%(test_data['Id'][i],p))

Gradient Descent for linear regression with tensorflow

In [ ]:
import tensorflow
import numpy
x_data = numpy.random.rand(100).astype(numpy.float32)
y_data = x_data * 0.1 + 0.3
W = tensorflow.Variable(tensorflow.random_uniform([1], -1.0, 1.0))
b = tensorflow.Variable(tensorflow.zeros([1]))
y = W * x_data + b

loss = tensorflow.reduce_mean(tensorflow.square(y-y_data))
optimizer = tensorflow.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)
init = tensorflow.initialize_all_variables()

sess = tensorflow.Session()
sess.run(init)

for step in range(201):
    sess.run(train)
    if (step % 20 == 0) :
        print(step, sess.run(W), sess.run(b))

Let's try tensorflow and do a LSTM for anomaly detection.

mnist with tensorflow

In [1]:
import tensorflow.examples.tutorials.mnist
mnist = tensorflow.examples.tutorials.mnist.input_data.read_data_sets('MNIST_data', one_hot=True)

import tensorflow as tensorflow
sess = tensorflow.InteractiveSession()

x = tensorflow.placeholder(tensorflow.float32, shape=[None, 784])
y_ = tensorflow.placeholder(tensorflow.float32, shape=[None, 10])

W = tensorflow.Variable(tensorflow.zeros([784,10]))
b = tensorflow.Variable(tensorflow.zeros([10]))

sess.run(tensorflow.initialize_all_variables())

y = tensorflow.nn.softmax(tensorflow.matmul(x, W) + b)

cross_entropy = tensorflow.reduce_mean(-tensorflow.reduce_sum(y_ * tensorflow.log(y), reduction_indices=[1]))

train_step = tensorflow.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

for i in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x:batch[0], y_:batch[1]})

correct_prediction = tensorflow.equal(tensorflow.argmax(y,1), tensorflow.argmax(y_,1))
accuracy = tensorflow.reduce_mean(tensorflow.cast(correct_prediction, tensorflow.float32))
print(accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
0.918


3 layer conv net

In [4]:
def weight_variable(shape):
    initial = tensorflow.truncated_normal(shape, stddev=0.1)
    return tensorflow.Variable(initial)
def bias_variable(shape):
    initial = tensorflow.constant(0.1, shape=shape)
    return tensorflow.Variable(initial)
def conv2d(x, W):
    return tensorflow.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
def max_pool_2x2(x):
    return tensorflow.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
x_image = tensorflow.reshape(x, [-1,28,28,1])
h_conv1 = tensorflow.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tensorflow.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tensorflow.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tensorflow.nn.relu(tensorflow.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tensorflow.placeholder(tensorflow.float32)
h_fc1_drop = tensorflow.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y_conv = tensorflow.nn.softmax(tensorflow.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy = tensorflow.reduce_mean(-tensorflow.reduce_sum(y_ * tensorflow.log(y_conv), reduction_indices=[1]))
train_step = tensorflow.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tensorflow.equal(tensorflow.argmax(y_conv,1), tensorflow.argmax(y_,1))
accuracy = tensorflow.reduce_mean(tensorflow.cast(correct_prediction, tensorflow.float32))
sess.run(tensorflow.initialize_all_variables())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))